# Aula 1: Comparando Regras com Modelo Supervisionado

- Comparar performance
- Supervisionado: LSTM
- Regras: VADER



---

### Pre processamento
- Classe Irrelevante vai virar Neutral
- Remover tweets com texto NaN



---

### VADER
- Criar uma coluna vader_sentiment
- Registrar Polarity Score nessa coluna
- Remover o score Compound
- Buscar o maior score entre Positivo, Negativo e Neutro
- Registrar na coluna vader_sentiment
- Avaliar a performancve comparando a coluna sentiment

# Aula 2: Regras VS Supervisionado I

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [12]:
url = 'https://raw.githubusercontent.com/AnaClaraGuerra22/PLN-LLMs-e-Gen-AI---Udemy/931c68630a663029c81bb1066b92d67ae90a4001/Analise%20de%20Sentimento/Data/Tweets2.csv'

tweets = pd.read_csv(url)
print(tweets.shape)
tweets.head()


(74682, 4)


,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [13]:
tweets['sentiment'].value_counts()

,count
sentiment,
Negative,22542
Positive,20832
Neutral,18318
Irrelevant,12990


In [16]:
tweets.loc[tweets['sentiment'] == 'Irrelevant', 'sentiment'] = 'Neutral'

In [17]:
tweets['sentiment'].value_counts()

,count
sentiment,
Neutral,31308
Negative,22542
Positive,20832


In [20]:
# remover com NaN
tweets = tweets.dropna(subset=['text'])
tweets.reset_index(drop=True, inplace=True)

In [21]:
tweets.shape

(73996, 4)

# Aula 3: Regras VS Supervisionado II

### Supervisionado

In [22]:
token = Tokenizer(num_words=1000)
token.fit_on_texts(tweets['text'].values) # cria o modelo

In [23]:
x = token.texts_to_sequences(tweets['text'].values) # executa o modelo
x = pad_sequences(x, padding = 'post', maxlen=100)    # deixar do mesmo tamanho as mensagens

In [24]:
Labelencoder = LabelEncoder()
y = Labelencoder.fit_transform(tweets['sentiment'])
print(y)

[2 2 2 ... 2 2 2]


In [25]:
y = to_categorical(y)
print(y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [27]:
x_train, x_test, y_tran, y_test = train_test_split(x, y, test_size=0.4)
x_test

array([[635,   1,  78, ...,   0,   0,   0],
       [322,  48, 265, ...,   0,   0,   0],
       [341, 315,  17, ...,   0,   0,   0],
       ...,
       [101, 101, 432, ...,   0,   0,   0],
       [ 33,   2, 120, ...,   0,   0,   0],
       [329,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [28]:
modelo = Sequential()

modelo.add(Embedding(input_dim=len(token.word_index), output_dim=128, input_length=x.shape[1]))

modelo.add(SpatialDropout1D(0.2))

modelo.add(LSTM(196, dropout=0.2, recurrent_dropout=0, activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True))

modelo.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [29]:
modelo.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(modelo.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [32]:
modelo.fit(x_train, y_tran, epochs=5, batch_size=500, verbose = True)

Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.4181 - loss: 1.0826
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.4213 - loss: 1.0816
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.4208 - loss: 1.0820
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.4156 - loss: 1.0837
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 156s 2s/step - accuracy: 0.4195 - loss: 1.0826


In [33]:
loss, accuracySupervisionado = modelo.evaluate(x_test, y_test)
print('Loss: ', loss)
print('Accuracy: ', accuracySupervisionado)

925/925 ━━━━━━━━━━━━━━━━━━━━ 65s 70ms/step - accuracy: 0.4144 - loss: 1.0841
Loss:  1.0824337005615234
Accuracy:  0.41852766275405884


In [34]:
prev = modelo.predict(x_test)
prev

925/925 ━━━━━━━━━━━━━━━━━━━━ 65s 70ms/step


array([[0.30111644, 0.4214764 , 0.2774072 ],
       [0.30111644, 0.4214764 , 0.2774072 ],
       [0.30111644, 0.4214764 , 0.2774072 ],
       ...,
       [0.3011164 , 0.42147636, 0.2774072 ],
       [0.30111644, 0.42147636, 0.2774072 ],
       [0.3011164 , 0.42147636, 0.2774072 ]], dtype=float32)

# Aula 4: Regras VS Supervisionado III

### VADER

In [35]:
instancia = SentimentIntensityAnalyzer()

In [36]:
tweets['vander_sentiment'] = ''

In [37]:
for y in range(len(tweets.index)):
  x = instancia.polarity_scores(tweets['text'].iloc[y])
  del x['compound']           # excluir o compound
  maior = max(x, key=x.get)  # pegar o maior score neg, pos ou neu
  tweets.loc[y,'vander_sentiment'] = maior


In [38]:
tweets['vander_sentiment'].value_counts

<bound method IndexOpsMixin.value_counts of 0        neu
1        neu
2        neu
3        neu
4        neu
        ... 
73991    neu
73992    neu
73993    neu
73994    neu
73995    neu
Name: vander_sentiment, Length: 73996, dtype: object>

In [39]:
tweets.loc[tweets['vander_sentiment'] == 'neu', 'vander_sentiment'] = 'Neutral'
tweets.loc[tweets['vander_sentiment'] == 'neg', 'vander_sentiment'] = 'Negative'
tweets.loc[tweets['vander_sentiment'] == 'pos', 'vander_sentiment'] = 'Positive'

In [40]:
tweets['vander_sentiment'].value_counts

<bound method IndexOpsMixin.value_counts of 0        Neutral
1        Neutral
2        Neutral
3        Neutral
4        Neutral
          ...   
73991    Neutral
73992    Neutral
73993    Neutral
73994    Neutral
73995    Neutral
Name: vander_sentiment, Length: 73996, dtype: object>

In [41]:
y_pred = tweets['vander_sentiment']
y_test = tweets['sentiment']

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 2004 19902   452]
 [ 1122 28384  1477]
 [  534 17295  2826]]


In [42]:
# medir a perfomance

accuracyVADER = accuracy_score(y_test, y_pred)
print(accuracyVADER)

0.44886210065408944


### Comparando acuracia entre os modelos

In [43]:
print('Acuracia do Supervisionado: ', accuracySupervisionado)
print('Acuracia do VADER: ', accuracyVADER)

Acuracia do Supervisionado:  0.41852766275405884
Acuracia do VADER:  0.44886210065408944
